# 2019 Biocuration EBI - Final Assessment

This assesment will test your skill in using Python to fetch, process, analyse and display data from **Ensembl**.

## Overview

The assessment is divided into four parts, each containing one or more _tasks_.

1. [Reading files and writing files](#part1): this part will test your understanding of how to __access and extract__ relevant information from a biological data file (3 tasks).
2. [Filtering with Ensembl REST API](#part2): this part measures your understanding of using Python to programmatically __query and download__ data from online databases/resources (2 tasks).
3. [Aggregating Pathway Information from a Database](#part3): this part tests your ability to construct and __execute valid queries__ of a local database using _SQL_ and _sqlite_ (1 task).
4. [Visualizing the Results](#part4): the final part tests your ability to __create summary plots__ of the data that you've collected (1 task).
    
All parts/tasks will test your ability to:

- read Python code
- work with variables, functions, modules, and different data types
- and construct well-organised, syntactically-valid code that can be easily maintained

## Exercise

You have been given a list of proteins which came up in a screen of human proteins that responded to a newly tested drug being developed by your collaborators. Those collaborators have turned to you, as someone familiar with programming in Python, and would like you to help them with two specific tasks:

1. __Filter__ the list of genes to include only those with orthologs in Mouse (which will be used for follow up testing the lab)
2. Predict which __biological pathways__ may be affected by this drug

<a id='part1'><h2>Part 1: Reading files and writing to files</h2></a>

You were given the results of the screen as a FASTA file of protein sequences, `Screen_results.fa`.

In [2]:
SCREEN_RESULTS_FILE = "Screen_results.fa"

In this file, each fasta sequence has an __Ensembl gene ID__ as part of their headers, which looks like this:

```
>ENSP00000487941.1 pep chromosome:GRCh38:7:142786213:142786224:1 gene:ENSG00000282431.1 transcript:ENST00000632684.1 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:TRBD1 description:T cell receptor beta diversity 1 [Source:HGNC Symbol;Acc:HGNC:12158]
```

Although the header is very long, the Ensembl Gene ID is only the small part of it beginning with "ENSG", and excluding the contents after the "." (`ENSG00000282431` in the example above.)

A colleague has written the code below, which reads the FASTA file and extracts the gene IDs from the header of each sequence. 

In [3]:
gid_list = []
with open(SCREEN_RESULTS_FILE, 'r') as fh:
    for line in fh.readlines():
        if line.startswith('>'):
            header = line.strip()
            header_fields = header.split()
            for field in header_fields:
                if field.startswith('gene:'):
                    gid = field[5:].split('.')[0]
                    gid_list.append(gid)

This piece of code works alright. However, you know that this is a process you're likely to want to repeat many more times in the future so it would be more sensible to encapsulate as a function.

### Task 1.1

__Complete the code in the block below to define a function__, `read_ensembl_ids`, which takes any FASTA file (for e.g. `SCREEN_RESULTS_FILE` in the code above) as input and return a list of gene IDs (for e.g. `gid_list` in the code above) taken from the sequence headers inside that file.

You should be able to get the returned list by using this line of code:

```python
gid_list = read_ensembl_ids(SCREEN_RESULTS_FILE)
```

Fill in the blanks in the code below. 

__A note on the assessment format__: Here and throughout the assessment, you are given template blocks of Python with blanks shown by placeholders (`...` and `+++`) which you should replace with your own working Python code.
 - `...` should be replaced by a single element of code. For example, these elements might be:
   - a function or variable name (e.g. `print`, `requests.get`, `listname.append`, etc)
   - a keyword (`for`, `in`, etc)
   - or an operator (`=`, `+`, etc)
 - `+++` should be replaced by _at least one_ complete line(s) of code

In [ ]:
# %load code_snippets/task_1_1.py

def read_ensembl_ids(...):
    +++
    return ...

gid_list = read_ensembl_ids(SCREEN_RESULTS_FILE)

### Task 1.2

1. Define a function that __writes all entries of a list to a file__. 
2. Call this function to write each entry of `gid_list` (from Task 1.1) to a file "GeneIDs.txt" in any order.

Fill in the blanks (`+++`) in the code below to complete the task.

**Reminder:** A docstring is written inside three quotes (e.g.`''' write your doctring here '''`) as the first statement of the function, and explains what your function does (its input, output, etc). This is useful as it gets printed when the `help` is used on the function.

In [ ]:
# %load code_snippets/task_1_2.py
def write_list_to_file(list_to_write, filename):
    '''
    Save the contents of a list, one entry per line, to a file.

    Arguments:
    - list_to_write:     a list of Ensembl Gene identifiers.
    - filename:          name of the file where the gene identifiers are stored.
    '''
    with open(filename, 'w') as out_fh: # 'w' stands for write
        +++

In [ ]:
# call function 
ENSG_ID_FILE = "GeneIDs.txt"
write_list_to_file(gid_list, ENSG_ID_FILE)

### Task 1.3

1. Check if the content in the file is correct by reading and printing it. 

(In order to read a flat file format, you can use `with open(filename, 'r') as in_fh:` (as in Tasks 1.1 & 1.2), where `r` stands for reading.)

In [ ]:
with ...
    +++

<a id='part2'><h2>Part 2: Filtering with Ensembl REST API</h2></a>

#### Ensembl documentation

- The [Ensembl REST wiki](https://github.com/Ensembl/ensembl-rest/wiki) has a lot of great documentation on using the Ensembl REST API.
- The [Ensembl REST documentation](https://rest.ensembl.org/) has a list of all of the "endpoints" known to the REST API.

### Task 2.1

The function below communicates with the **Ensembl Homologues REST endpoint**. The Ensemble REST API can return results in many formats, including the [JSON format](https://www.json.org). The Python **requests** module includes functionality to decode a JSON object into Python objects (lists and dictionaries).

1. Read more about the *homology* endpoint (see the [3rd Example Request](https://rest.ensembl.org/documentation/info/homology_ensemblgene) and [example file](https://rest.ensembl.org/homology/id/ENSG00000157764?target_taxon=10090;target_species=cow;type=orthologues;sequence=cdna;content-type=application/json)) for reference, which uses the gene id 'ENSG00000157764' and taxon '10090'. Pay particular attention to these flags: *target_taxon*, *type* and *sequence*.
2. Finish the function below to make it return a decoded JSON object with information on homologues for a given Gene ID, in a given taxon.

**Note:** If there are no homologues in the target taxon, the Ensembl Homologues endpoint returns a single dictionary, with the *key* "data", and an empty list as its *value*.

Fill in the blanks (`...`) in the code below to complete the task.

In [ ]:
# %load code_snippets/task_2_1.py
import sys
import requests

def fetch_ensembl_homologues(gid, target_taxon='10090'):
    '''
    Retrieves Gene IDs of homologous proteins from the Ensembl
    Homology REST endpoint for a given taxon. 
    
    Arguments:
    - gid:                Ensembl Gene ID.
    - target_taxon:       taxonomic identifier of the species. 
                          (default: 10090 [mouse])
                          
    Returns:
    - The decoded JSON response from the Ensembl Homology endpoint
    '''
    server = "http://rest.ensembl.org"
    ext = "/homology/id/{0}?target_taxon={1};type=orthologues;sequence=cdna"
    ext = ...
    response = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    if not response.ok:
        response.raise_for_status()
        sys.exit(1)
    decoded = response.json()
    return decoded

We will now call the function in the code snippet below, using it to get a decoded JSON object for every gene in the list of IDs that you generated earlier.

Fill in the blanks (`...` and `+++`) in the code below to complete the task.

In [ ]:
# %load code_snippets/task_2_1_b.py
tax_id = '10090'
SCREEN_RESULTS_FILE = "Screen_results.fa"
# call function "read_ensembl_ids"
gid_list = ...
for gid in gid_list:
    # call function "fetch_ensembl_homologues"
    decoded_file = ...

### Task 2.2

In order to identify candidates for future study in the lab, filter the list of human genes to retain a list of those which have an ortholog in mouse, according to Ensembl.

1. Write a function that uses the genes listed in `GeneIDs.txt` based on whether the gene transcript has an ortholog in mouse, with over 50% sequence identity. 
2. Use the function that retrieves JSON file (from Task 2.1) inside this function to get the ortholog and percentage identity information.

#### Hints

- Look carefully at the structure of the JSON data returned by the API. Where is the *percentage identity* stored in the output?
- Please ignore the missing homology information for a few Ensembl IDs.

Fill in the blanks (`...` and `+++`) in the code below to complete the task.

In [ ]:
%load code_snippets/task_2_2.py
def filter_ortholog_by_identity(gene_list, 
                                target_taxon='10090', 
                                min_perc_ident=50.0):
    '''
    Creates a list of Ensembl genes by the presence of its ortholog in a species, 
    and filters it by a threshold defined for percentage identity of its sequence.
    
    Arguments:
    - gene_list:         a list of Ensembl gene id
    - target_taxon:      taxonomic identifier of the species in which to look
                         for orthologs. (default: 10090 [mouse])
    - min_perc_ident:    minimum percentage identity for retaining an ortholog. 
                         (default: 50.0)
    
    Returns:
    - a list of Gene ID's as strings
    '''
    filtered_gene_list = []
    for gene in gene_list:
        decoded = ...
        for homolog in decoded['data'][0]['homologies']:
            perc_id = ...
            if ...:
                +++
    filtered_gene_list = list(set(filtered_gene_list)) # remove duplicates
    return filtered_gene_list

In [ ]:
# Hint: Store the contents of the file "GeneIDs.txt" in a new list
# called 'gene_list', where each element is a single Gene ID

ENSG_ID_FILE = "GeneIDs.txt"
gene_list = []
with ...
    +++

In [ ]:
# use gene_list to call function: `filter_ortholog_by_identity`
filtered_ortholog_list = filter_ortholog_by_identity(gene_list)
print("keeping {0} out of {1} IDs".format(len(filtered_ortholog_list), len(gene_list)))

Write the list of orthologs, filtered by percentage identity above 50, to a file "GeneIDs_with_orthologs.txt" (Hint: see Task 1.2). _The list of GeneIDs must be unique, and may be in any order._

In [ ]:
ENSG_ORTHOLOG_FILE = "GeneIDs_with_orthologs.txt"
write_list_to_file(filtered_ortholog_list, ENSG_ORTHOLOG_FILE)

<a id='part3'><h2>Part 3: Aggregating Pathway Information from a Database</h2></a>



[Reactome](https://reactome.org/) is a database of manually-curated pathway annotations for different genes.

Your collaborators have provided a **Sqlite** database with mappings between Ensembl Gene ID's and Reactome annotations. The database is contained in the file **reactome.db**, which contains a single table called *ensembl_reactome* which has the following columns:

* *gene_id*: The Ensembl Gene ID (e.g.: ENSG0000016650)
* *reactome_id*: The Reactome ID (e.g.: R-HSA-73857)
* *reactome_name*: The (human readable) Reactome term name (e.g.: RNA Polymerase II Transcription)

**Note:** Each Ensembl Gene ID may have multiple Reactome terms associated with it.

Use the **reactome.db** database, and Python's **sqlite3** package, create a list of how often each pathway is present in a Gene which came out in the screen (after filtering for orthologs. You will need to store _paired information_ about the pathways' described references: the pathways found and a count of the number of times that each pathway was seen.

Once you have extracted this information from the database,
__write the results to a file__ called `Reactome_counts.tsv`. 
The file should have two columns,
with information for one pathway per line.
The first field in a row should contain the name of a pathway,
and the second the frequency count for that pathway.
_The results can be listed in the file in any order._

### Task 3

1. __Retrieve a list of Reactome Pathway names__ from a **local Sqlite databse** and write them as a tab separated file of Reactome ID's + count.


#### Documentation

- The [Python Central sqlite tutorial](https://www.pythoncentral.io/introduction-to-sqlite-in-python/) is a great tutorial on using Sqlite with python

Fill in the blanks (`...` and `+++`) in the code below to complete the task.

In [ ]:
# %load code_snippets/task_3_1.py

import sqlite3

def find_reactome_annotation_counts(gene_list, db_file):
    '''
    Find the reactome annotations for a list of gene ids from a database.
    
    Arguments:
    - gene_list:   a list of Ensembl Gene identifiers to search for.
    - db_file:     the file name of the Sqlite database to use.
    
    Returns:
    - a dictionary with identifiers from the target database as keys 
        and a count of the number of times that identifier was
        observed as values.
    '''
    
    # establish a connection and a cursor to the database file
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    reactome_counts = {} # initializing an empty dictionary
    for gene_id in gene_list:

        # Select all reactome_name values for this gene_id
        query = ...
        
        cursor.execute(query, (gene_id, ))
        
        for result in cursor.fetchall():
            name = result[0]
            if name not in reactome_counts:
                reactome_counts[name] = 1
            else:
                +++
    return reactome_counts

In [ ]:
# %load code_snippets/task_3_2.py
# read filtered IDs from last step

ENSG_ORTHOLOG_FILE = "GeneIDs_with_orthologs.txt"
gene_list = []
with open(ENSG_ORTHOLOG_FILE, 'r') as in_fh: # 'r' stands for write
    +++
        
reactome_counts = find_reactome_annotation_counts(gene_list, "reactome.db")


In [ ]:
# %load code_snippets/task_3_3.py
# Hint: Save the Reactome results to a file, where each line has the Reactome pathway name, and the
# number of times it occurs, separated by a 'tab' ('\t') character
ENSG_ID_REACTOME_FILE = "Reactome_counts.tsv"

with ...
    +++

## Part 4: Visualizing the Results

### Task 4

__Make a bar plot__ of the 10 most frequently occuring Reactome groups.

Identifying the __Reactome pathways__ that are potentially affected by the drug that our collabotors have screened, will indicate to the different __biological processes__ that might be altered by this drug. 

Finding a good way to represent or _visualize such information_ will make interpreting the results much easier.

The **matplotlib** library has a lot of useful plotting functions. Use this library to create a bar plot of the top 10 most frequently-occuring Reactome terms in the results from Part 3.
Save the plot to a file called *plot.png*.

**Hints**

- The [`bar`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.bar.html) and [`barh`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.barh.html) functions in **matplotlib** can both be used to create bar plots.
- Use the **jupyter** "magic" command `%matplotlib inline` on the top of your script to show plots directly in the **jupyter notebook**.

Fill in the blanks (`...` and `+++`) in the code below to complete the task.

In [ ]:
# %load code_snippets/task_4_1.py
%matplotlib inline

import matplotlib.pyplot as plt

def plot_cross_refs(results, limit=10):
    '''Draw a horizontal bar plot of results from cross-referencing,
    limited to the `limit` terms found most frequently.
    
    Arguments:
    - results: a dictionary of results to plot.
                 Keys should be database identifiers/terms, with values
                 as frequency counts for these respective identifiers.
    - limit:     the number of top-ranked identifiers/terms (by 
                 frequency) to plot. (default: 10)
    - title:     a title for the plot.
    
    Note: If multiple xrefs which rank at the limit (e.g., 3 xrefs with
    the same score values at rank 10), all of them will be displayed. The final
    plot may have more than `limit` values.
    '''
    
    all_counts = results.values()
    threshold = sorted(all_counts, reverse=True)[:limit][-1]

    terms = []
    counts = []

    for key, val in results.items():
       +++
    
    # Horizontal bar plots can call matplotlib functions using 'plt.' to create the plot here
    
            
    plt.barh(list(range(len(counts))),
        counts,tick_label=terms)
    
    # save plot to a file
    +++

In [ ]:
# %load code_snippets/task_4_2.py
# create input for the function
ENSG_ID_REACTOME_FILE = "Reactome_counts.tsv"
reactome_count = {}
with open(ENSG_ID_REACTOME_FILE, 'r') as in_fh:
    +++

In [ ]:
# call function
plot_cross_refs(reactome_count, limit=10)